## Importaciones de las librerías y paquetes necesarios

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import models
import torchvision.models.segmentation as segmentation
import torch.nn as nn
from torch.optim import Adam
import torchvision
from albumentations.pytorch import ToTensorV2
import albumentations as A
import os
from tqdm import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import pandas as pd
import pickle
import csv

## Creación de la clase heredada de Dataset

In [ ]:
TRAIN_IMAGES_PATH = "G:/dataset_rural_y_urbano/train"
TRAIN_MASKS_PATH = "G:/dataset_rural_y_urbano/train_masks"

In [ ]:
class SemanticSegmentation(Dataset):
    def __init__(self,img_dir, mask_dir, transform = None):
        self.transforms = transform
        self.images = sorted(os.listdir(img_dir))
        self.masks = sorted(os.listdir(mask_dir))            
            
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self,index):
        img = np.array(Image.open(TRAIN_IMAGES_PATH + "/" + self.images[index]))
        mask = np.array(Image.open(TRAIN_MASKS_PATH + "/" + self.masks[index]))
        if self.transforms is not None:
            transforms = self.transforms(image=img,mask=mask)
            img = transforms['image']
            mask = transforms['mask']
            mask = torch.max(mask,dim=2)[0]
        return img, mask

## Obtención de los conjuntos de entrenamiento, validación y testeo

In [ ]:
def get_images(image_dir, mask_dir, transform = None, batch_size=1, shuffle=True, pin_memory=True):
    
    semantic_segmentation = SemanticSegmentation(image_dir, mask_dir, transform = transform)
    
    train_size = int(0.8 * len(semantic_segmentation))
    val_size = int(0.1 *len(semantic_segmentation))
    test_size = len(semantic_segmentation) - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(semantic_segmentation, [train_size, val_size, test_size])
    
    train_set = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    val_set = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    test_set = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    
    return train_set, val_set, test_set

In [ ]:
transforms = A.Compose([
                        A.Resize(256,480),
                        ToTensorV2()
                        ])

In [ ]:
train_set, val_set, test_set = get_images(TRAIN_IMAGES_PATH, TRAIN_MASKS_PATH, transform=transforms, batch_size=13)

## Para almacenar los distintos conjuntos de entrenamiento, validación y testeo (si ya se tienen los conjuntos, se salta este paso y se cargan en el siguiente)

In [ ]:
# To save the training set
with open('G:/dataset_rural_y_urbano/saved_sets/train_set_dataset_rural_y_urbano.pkl', 'wb') as f:
    pickle.dump(train_set, f)

# To save the validation set
with open('G:/dataset_rural_y_urbano/saved_sets/val_set_dataset_rural_y_urbano.pkl', 'wb') as f:
    pickle.dump(val_set, f)
    
# To save the test set
with open('G:/dataset_rural_y_urbano/saved_sets/test_set_dataset_rural_y_urbano.pkl', 'wb') as f:
    pickle.dump(test_set, f)

## Se pueden cargar los conjuntos si ya se tienen

In [ ]:
with open('G:/dataset_rural_y_urbano/saved_sets/train_set_dataset_rural_y_urbano.pkl', 'rb') as f:
    train_set = pickle.load(f)
    
with open('G:/dataset_rural_y_urbano/saved_sets/train_set_dataset_rural_y_urbano.pkl', 'rb') as f:
    val_set = pickle.load(f)
    
with open('G:/dataset_rural_y_urbano/saved_sets/train_set_dataset_rural_y_urbano.pkl', 'rb') as f:
    test_set = pickle.load(f)

In [ ]:
print(len(train_set))
print(len(val_set))
print(len(test_set))

## Elección del dispositivo de entrenamiento

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

   ### Opción 1) Se carga el modelo entrenado con el dataset rural y se se modifica la última clase

In [ ]:
model = torch.load('G:/dataset_rural/saved_models/modelo_guardado_dataset_rural_DeepLabV3_Epoch_23.pt')
model = model.to('cuda')

num_new_classes = 14

model.classifier[-1] = nn.Conv2d(256, num_new_classes, kernel_size=(1, 1), stride=(1, 1))
model.aux_classifier[-1] = nn.Conv2d(256, num_new_classes, kernel_size=(1, 1), stride=(1, 1))

   ### Opción 2) Se carga el modelo entrenado con el dataset rural pero con las clases que no están en el dataset urbano puestas como clase clutter, y se se modifica la última clase

In [ ]:
model = torch.load('G:/dataset_rural/saved_models/modelo_guardado_dataset_rural_con_clutter_DeepLabV3_Epoch_20.pt')
model = model.to('cuda')

num_new_classes = 9

model.classifier[-1] = nn.Conv2d(256, num_new_classes, kernel_size=(1, 1), stride=(1, 1))
model.aux_classifier[-1] = nn.Conv2d(256, num_new_classes, kernel_size=(1, 1), stride=(1, 1))

## Definiendo hiperparámetros y opciones del entrenamiento

In [ ]:
LEARNING_RATE = 1e-4
num_epochs = 50

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

## Función que devuelve a partir de una predicción la matriz con el tamaño original de la máscara

In [ ]:
def originalMatrix(arr):
    if isinstance(arr, torch.Tensor):
        arr = arr.cpu().numpy()
        
    # Convert the original matrix to a matrix with zeros in the columns of G and B
    converted_matrix = np.zeros(arr.shape + (3,))
    
    # Copy the values of R into the column of R
    converted_matrix[..., 0] = arr  
    
    return converted_matrix

## Función que pasa la máscara a los colores originales

In [ ]:
def originalColorMask(converted_matrix):
    
    # New colors
    new_colors = np.array([
        [1, 0, 0],
        [2, 0, 0],
        [3, 0, 0],
        [4, 0, 0],
        [5, 0, 0],
        [6, 0, 0],
        [7, 0, 0],
        [8, 0, 0],
        [9, 0, 0],
        [10, 0, 0],
        [11, 0, 0],
        [12, 0, 0],
        [13, 0, 0]
    ])

    # Originals colors
    original_colors = np.array([
        [127, 127, 0],
        [0, 255, 0],
        [0, 127, 0],
        [255, 255, 0],
        [255, 127, 0],
        [255, 255, 255],
        [255, 0, 255],
        [127, 127, 127],
        [0, 0, 255],
        [0, 255, 255],
        [127, 127, 63],
        [255, 0, 0],
        [0, 0, 0]
    ])

    original_arr = converted_matrix.astype(int)

    # Crea una nueva matriz NumPy con los nuevos colores
    new_colors_arr = np.zeros_like(original_arr)
    for i in range(len(original_colors)):
        mask = np.all(original_arr == new_colors[i], axis=-1)
        new_colors_arr[mask] = original_colors[i]

    # Convierte la matriz NumPy de nuevo a una imagen y guarda en el directorio de salida
    new_image = Image.fromarray(new_colors_arr.astype(np.uint8))
    return new_image

## Función que pasa la predicción a los colores originales

In [ ]:
def originalColorPrediction(converted_matrix):
    
    # Originals colors
    original_colors = np.array([
        [127, 127, 0],
        [0, 255, 0],
        [0, 127, 0],
        [255, 255, 0],
        [255, 127, 0],
        [255, 255, 255],
        [255, 0, 255],
        [127, 127, 127],
        [0, 0, 255],
        [0, 255, 255],
        [127, 127, 63],
        [255, 0, 0],
        [0, 0, 0]
    ])
    
    # New colors
    new_colors = np.array([
        [1, 0, 0],
        [2, 0, 0],
        [3, 0, 0],
        [4, 0, 0],
        [5, 0, 0],
        [6, 0, 0],
        [7, 0, 0],
        [8, 0, 0],
        [9, 0, 0],
        [10, 0, 0],
        [11, 0, 0],
        [12, 0, 0],
        [13, 0, 0]
    ])


    original_arr = converted_matrix.astype(int)[0]

    # Create a new NumPy array with the new colors
    new_colors_arr = np.zeros_like(original_arr)
    for i in range(len(original_colors)):
        mask = np.all(original_arr == new_colors[i], axis=-1)
        new_colors_arr[mask] = original_colors[i]

    # Convert the NumPy array back to an image
    new_image = Image.fromarray(new_colors_arr.astype(np.uint8))
    return new_image

## Se definen las funciones que calcula IOU y DICE durante el entrenamiento y validación

In [ ]:
def calculate_iou_training(img1, img2):
    
    img1 = np.asarray(img1)
    img2 = np.asarray(img2)
    
    # We calculate the intersection between the two pixel matrices
    intersection = np.all(img1 == img2, axis=-1)

    # We count the total pixels that intersect
    total_intersection = np.count_nonzero(intersection)
    
    # We count the total number of pixels in both images
    total_pixels = img1.shape[0] * img1.shape[1] + img2.shape[0] * img2.shape[1]
    
    # We calculate the union
    union = total_pixels - total_intersection
    
    return total_intersection / union

In [ ]:
def dice_coef_training(im1, im2):
        
    img1 = np.asarray(im1)
    img2 = np.asarray(im2)
    
    # We calculate the intersection between the two pixel matrices
    intersection = np.all(img1 == img2, axis=-1)

    # We count the total pixels that intersect
    total_intersection = np.count_nonzero(intersection)
    
    width, height = im1.size
    num_pixeles_im1 = width * height
    
    width, height = im2.size
    num_pixeles_im2 = width * height
    
    # Calculate the DICE coefficient
    dice = (2. * total_intersection) / (num_pixeles_im1 + num_pixeles_im2)
    
    return dice

## Función que devuelve el valor de IOU y DICE en función de TP, FP  y FN

In [ ]:
def calculate_metrics(colorMaskOriginal, colorPredsOriginal):

    count = {}

    equivalence_colors = {
        (127, 127, 63):"Hill", 
        (0, 127, 0):"Forest",
        (255, 255, 0):"Residential",
        (0, 255, 0):"Land",
        (255, 0, 255):"Church",
        (255, 255, 255):"Road",
        (127, 127, 0):"Fence",
        (255, 0, 0):"Person",
        (127, 127, 127):"Car",
        (0, 255, 255):"Sky",
        (255, 127, 0):"Haystack",
        (0, 0, 255):"Water",
        (0, 0, 0):"Clutter",
    }
    


    for rgb, name in equivalence_colors.items():
        count[rgb] = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}


    for a in range(colorMaskOriginal.width):
        for b in range(colorMaskOriginal.height):
            for category, color in equivalence_colors.items():
                if colorPredsOriginal.getpixel((a,b)) == category and colorMaskOriginal.getpixel((a,b)) == category:
                    count[category]['TP'] += 1
                elif colorPredsOriginal.getpixel((a,b)) != category and colorMaskOriginal.getpixel((a,b)) != category:
                    count[category]['TN'] += 1
                elif colorPredsOriginal.getpixel((a,b)) == category and colorMaskOriginal.getpixel((a,b)) != category:
                    count[category]['FP'] += 1
                elif colorPredsOriginal.getpixel((a,b)) != category and colorMaskOriginal.getpixel((a,b)) == category:
                    count[category]['FN'] += 1   
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0 
    for category, data in count.items():
        tp += data["TP"]
        fp += data["FP"]
        tn += data["TN"]
        fn += data["FN"]
      
    valueIOU = tp / (fp + tp + fn)
    valueDICE = (2 * tp) / (fp + (2 * tp) + fn)
        
    return valueIOU, valueDICE

## Función de validación para el entrenamiento

In [ ]:
def validation(model, loader):
    correct = 0
    total = 0
    cost = 0.    
    dice = 0.
    iou = 0.
    images_count = 0
    
    model = model.to(DEVICE)

    with torch.no_grad():
        for x, y in loader:
            
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            y = y.type(torch.long)
                        
            x = x.float()
            preds = model(x)
            
            softmax = nn.Softmax(dim=1)
            predictions = torch.argmax(softmax(model(x.float())['out']), axis=1).to(DEVICE)
            
            cost += loss_fn(preds, y)            
        
            correct += (predictions == y).sum().item()
            total += y.numel()
            
            for i in range(x.shape[0]): 
                
                prediction = predictions[i]
                prediction = prediction.unsqueeze(0)

                mask = y[i].squeeze().cpu().numpy()

                matrizPredsOriginal = originalMatrix(prediction)
                colorPredsOriginal = originalColorPrediction(matrizPredsOriginal)

                matrizMaskOriginal = originalMatrix(mask)
                colorMaskOriginal = originalColorMask(matrizMaskOriginal)
                
                dice += dice_coef_training(colorPredsOriginal, colorMaskOriginal)         
                iou += calculate_iou_training(colorPredsOriginal, colorMaskOriginal)
                
                images_count += 1
     
                
        return cost/len(loader), correct/total, dice/images_count, iou/images_count

## Entrenamiento del modelo

In [ ]:
store_every = 33
early_stop = False
best_valid_loss = np.Inf
epochs_no_improve = 0
max_epochs_stop = 3

model = model.to('cuda')


validation_costs = []

for epoch in range(num_epochs):
    
    mb = 0
    train_cost_acum = 0.
    dice_train = 0.
    iou_train = 0.
    count_images = 0

    loop = tqdm(enumerate(train_set),total=len(train_set))
    for batch_idx, (data, targets) in loop:
        data = data.to(DEVICE, dtype=torch.float32)

        targets = targets.to(DEVICE)
        targets = targets.type(torch.long)
        
        # Forward
        with torch.cuda.amp.autocast():
            data = data.half()                           # Esta línea la he puesto yo
            predictions = model(data)['out']
            loss = loss_fn(predictions, targets) 
            
            softmax = nn.Softmax(dim=1)
            predictions = torch.argmax(softmax(predictions),axis=1).to(DEVICE)
            
        # Backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Update tqdm loop
        loop.set_postfix(loss=loss.item())

        mb += 1
        train_cost_acum += loss.item()

        # Calculate IOU and DICE values from the current training batch
        for i in range(data.shape[0]):
            with torch.no_grad():
                prediction = predictions[i]
                prediction = prediction.unsqueeze(0)


            mask = targets[i].squeeze().cpu().numpy()     

            # Both the mask and the prediction are obtained with the original colors
            matrizPredsOriginal = originalMatrix(prediction)
            colorPredsOriginal = originalColorPrediction(matrizPredsOriginal)

            matrizMaskOriginal = originalMatrix(mask)
            colorMaskOriginal = originalColorMask(matrizMaskOriginal)

            dice_train += dice_coef_training(colorPredsOriginal, colorMaskOriginal)         
            iou_train += calculate_iou_training(colorPredsOriginal, colorMaskOriginal)

            count_images += 1

        # The results are displayed and the model is stored
        if mb%store_every == 0:
            val_cost, val_acc, dice, iou = validation(model, val_set)
            train_cost_every = float(train_cost_acum)/mb
            
            validation_costs.append(val_cost)

            print(f'epoch: {epoch}, train cost: {train_cost_every:.4f}, val cost: {val_cost:.4f}, val acc: {val_acc:.4f},'
                    f'train dice: {dice_train/count_images}, train iou: {iou_train/count_images}, val dice: {dice}, val iou: {iou}')

            file_name = "G:/transfer_learning/saved_models/modelo_guardado_transfer_learning_Epoch_" + str(epoch) + ".pt"
            torch.save(model, file_name)
            
            if val_cost < best_valid_loss:
                best_valid_loss = val_cost
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

                # If the validation cost does not improve by max_epochs_stop, stop the training
                if epochs_no_improve == max_epochs_stop:
                    early_stop = True
                    print("Early stopping!")
                    print("La época con el coste de validación más bajo es la época: " + str(epoch - 3))
                    break
                    
        if epochs_no_improve == max_epochs_stop:
            break

## Para cargar el mejor modelo una vez que se haya concluido el entrenamiento

In [ ]:
model = torch.load('G:/transfer_learning/saved_models/modelo_guardado_transfer_learning_Epoch_13.pt')

## Obtención de los TP, TN, FP y FN para el conjunto de testeo

In [ ]:
container_values = {}
model = model.to('cpu')

color_equivalence = {
        '(127, 127, 63)':"Hill", 
        '(0, 127, 0)':"Forest",
        '(255, 255, 0)':"Residential",
        '(0, 255, 0)':"Land",
        '(255, 0, 255)':"Church",
        '(255, 255, 255)':"Road",
        '(127, 127, 0)':"Fence",
        '(255, 0, 0)':"Person",
        '(127, 127, 127)':"Car",
        '(0, 255, 255)':"Sky",
        '(255, 127, 0)':"Haystack",
        '(0, 0, 255)':"Water",
        '(0, 0, 0)':"Clutter",
}


for rgb, name in color_equivalence.items():
    container_values[rgb] = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0, 'IOU':0, 'total':0, 'coincidentes':0}

    
for x1, y in test_set:
    
 
    # The TP, TN, FP and FN of the first 4 images of the current batch are obtained
    x = x1.float().to('cpu')[0:4]
    predictions = model(x)
    targets = y[0:4]
    softmax = nn.Softmax(dim=1)
    preds = torch.argmax(softmax(predictions['out']), axis=1).to('cpu')

    for i in range(x.shape[0]):  
        with torch.no_grad():
            prediction = preds[i].unsqueeze(0)
            mask = targets[i].squeeze().cpu().numpy()

            matrizPredsOriginal = originalMatrix(prediction)
            colorPredsOriginal = originalColorPrediction(matrizPredsOriginal)

            matrizMaskOriginal = originalMatrix(mask)
            colorMaskOriginal = originalColorMask(matrizMaskOriginal)

            for i in range(colorMaskOriginal.width):
                for j in range(colorMaskOriginal.height):
                    mask_color = str(colorMaskOriginal.getpixel((i,j)))
                    container_values[mask_color]['total'] += 1
                    for category, color in color_equivalence.items():
                        if str(colorPredsOriginal.getpixel((i,j))) == str(category) and str(colorMaskOriginal.getpixel((i,j))) == str(category):
                            container_values[category]['TP'] += 1
                            container_values[category]['coincidentes'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) != str(category) and str(colorMaskOriginal.getpixel((i,j))) != str(category):
                            container_values[category]['TN'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) == str(category) and str(colorMaskOriginal.getpixel((i,j))) != str(category):
                            container_values[category]['FP'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) != str(category) and str(colorMaskOriginal.getpixel((i,j))) == str(category):
                            container_values[category]['FN'] += 1
                            
                            
    # The TP, TN, FP and FN of the next 4 images of the current batch are obtained                       
    x = x1.float().to('cpu')[4:8]
    predictions = model(x)

    targets = y[4:8]
    softmax = nn.Softmax(dim=1)
    preds = torch.argmax(softmax(predictions), axis=1).to('cpu')

    for i in range(x.shape[0]):  
        with torch.no_grad():
            prediction = preds[i].unsqueeze(0)
            mask = targets[i].squeeze().cpu().numpy()

            matrizPredsOriginal = originalMatrix(prediction)
            colorPredsOriginal = originalColorPrediction(matrizPredsOriginal)

            matrizMaskOriginal = originalMatrix(mask)
            colorMaskOriginal = originalColorMask(matrizMaskOriginal)

            for i in range(colorMaskOriginal.width):
                for j in range(colorMaskOriginal.height):
                    mask_color = str(colorMaskOriginal.getpixel((i,j)))
                    container_values[mask_color]['total'] += 1
                    for category, color in color_equivalence.items():
                        if str(colorPredsOriginal.getpixel((i,j))) == str(category) and str(colorMaskOriginal.getpixel((i,j))) == str(category):
                            container_values[category]['TP'] += 1
                            container_values[category]['coincidentes'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) != str(category) and str(colorMaskOriginal.getpixel((i,j))) != str(category):
                            container_values[category]['TN'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) == str(category) and str(colorMaskOriginal.getpixel((i,j))) != str(category):
                            container_values[category]['FP'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) != str(category) and str(colorMaskOriginal.getpixel((i,j))) == str(category):
                            container_values[category]['FN'] += 1
                 
    
    # The TP, TN, FP and FN of the last 4 images of the current batch are obtained
    x = x1.float().to('cpu')[8:13]
    predictions = model(x)

    targets = y[8:13]
    softmax = nn.Softmax(dim=1)
    preds = torch.argmax(softmax(predictions), axis=1).to('cpu')

    for i in range(x.shape[0]):  
        with torch.no_grad():
            prediction = preds[i].unsqueeze(0)
            mask = targets[i].squeeze().cpu().numpy()

            matrizPredsOriginal = originalMatrix(prediction)
            colorPredsOriginal = originalColorPrediction(matrizPredsOriginal)

            matrizMaskOriginal = originalMatrix(mask)
            colorMaskOriginal = originalColorMask(matrizMaskOriginal)

            for i in range(colorMaskOriginal.width):
                for j in range(colorMaskOriginal.height):
                    mask_color = str(colorMaskOriginal.getpixel((i,j)))
                    container_values[mask_color]['total'] += 1
                    for category, color in color_equivalence.items():
                        if str(colorPredsOriginal.getpixel((i,j))) == str(category) and str(colorMaskOriginal.getpixel((i,j))) == str(category):
                            container_values[category]['TP'] += 1
                            container_values[category]['coincidentes'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) != str(category) and str(colorMaskOriginal.getpixel((i,j))) != str(category):
                            container_values[category]['TN'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) == str(category) and str(colorMaskOriginal.getpixel((i,j))) != str(category):
                            container_values[category]['FP'] += 1
                        elif str(colorPredsOriginal.getpixel((i,j))) != str(category) and str(colorMaskOriginal.getpixel((i,j))) == str(category):
                            container_values[category]['FN'] += 1

## Guardado del diccionario con los valores de TP, TN FP y FN

In [ ]:
with open('G:/transfer_learning/saved_values/test_transfer_learning.csv', 'w') as archive:
    writer = csv.writer(archive)
    for category, value in container_values.items():
        writer.writerow([category, value])

## Obtención de las métricas a partir de los TP, TN, FP y FN

In [ ]:
tp_global = 0
fp_global = 0
tn_global = 0
fn_global = 0

table = PrettyTable()
table.field_names = ["Clase", "Nombre Clase", "Precisión", "Recall", "Accuracy", "F1", "IOU"]

for category, data in container_values.items():
    tp = data["TP"]
    fp = data["FP"]
    tn = data["TN"]
    fn = data["FN"]
    
    tp_global += data["TP"]
    fp_global += data["FP"]
    tn_global += data["TN"]
    fn_global += data["FN"]

    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    f1 = 2*((precision*recall)/(precision+recall))
    IOU_value = tp / (fp + tp + fn)
    
    
    class_name = color_equivalence[category]
    container_values[category]['IOU'] = IOU_value
    
    
    table.add_row([category, class_name, f"{precision:.4f}", f"{recall:.4f}", f"{accuracy:.4f}", f"{f1:.4f}", f"{IOU_value:.4f}"])


precision_global = tp_global / (tp_global + fp_global)
recall_global = tp_global / (tp_global + fn_global)
accuracy_global = (tp_global + tn_global) / (tp_global + tn_global + fp_global + fn_global)
f1_global = 2*((precision_global*recall_global)/(precision_global+recall_global))
iou_global = tp_global / (fp_global + tp_global + fn_global)

table.add_row(["", "Global", f"{precision_global:.4f}", f"{recall_global:.4f}", f"{accuracy_global:.4f}", f"{f1_global:.4f}", f"{iou_global:.4f}"])

print(table)

## Obtención de una predicción de las imágenes de testeo

In [ ]:
color_equivalence = {
        '(127, 127, 63)':"Hill", 
        '(0, 127, 0)':"Forest",
        '(255, 255, 0)':"Residential",
        '(0, 255, 0)':"Land",
        '(255, 0, 255)':"Church",
        '(255, 255, 255)':"Road",
        '(127, 127, 0)':"Fence",
        '(255, 0, 0)':"Person",
        '(127, 127, 127)':"Car",
        '(0, 255, 255)':"Sky",
        '(255, 127, 0)':"Haystack",
        '(0, 0, 255)':"Water",
        '(0, 0, 0)':"Clutter",
}


model = model.to('cpu')
for x,y in test_set:
    x = x.to('cpu')
    fig , ax =  plt.subplots(1, 3, figsize=(18, 18))
    softmax = nn.Softmax(dim=1)
    preds = torch.argmax(softmax(model(x.float())['out']), axis=1).to('cpu')
    img1 = np.transpose(np.array(x[0,:,:,:].to('cpu')),(1,2,0))
    preds1 = np.array(preds[0,:,:])
    mask1 = np.array(y[0,:,:])
    
    
    ax[0].set_title('Image')
    ax[1].set_title('Prediction')
    ax[2].set_title('Mask')
    
    ax[0].axis("off")
    
    # Loop through all images in the current batch
    for i in range(x.shape[0]):
        
        # Get the prediction for the current image
        prediction = preds[i]
        prediction = prediction.unsqueeze(0)

        mask = y[i].squeeze().cpu().numpy()
        
        matrizPredsOriginal = originalMatrix(prediction)
        colorPredsOriginal = originalColorPrediction(matrizPredsOriginal)

        matrizMaskOriginal = originalMatrix(mask)
        colorMaskOriginal = originalColorMask(matrizMaskOriginal)
        
        break
    
    ax[0].imshow(img1)
    ax[1].imshow(colorPredsOriginal)
    ax[2].imshow(colorMaskOriginal)  
    
    break


# Create the dictionary to store the values
container_values = {}

# Loop through the pixels of the actual image and count the pixels of each color
for i in range(colorMaskOriginal.width):
    for j in range(colorMaskOriginal.height):
        color = colorMaskOriginal.getpixel((i, j))
        if color in container_values:
            container_values[color]["total"] += 1
        else:
            container_values[color] = {"total": 1, "coincidentes": 0}

# Loop through the pixels of both images and compare their RGB value
for i in range(colorMaskOriginal.width):
    for j in range(colorMaskOriginal.height):
        real_color = colorMaskOriginal.getpixel((i, j))
        preds_color = colorPredsOriginal.getpixel((i, j))
        if real_color == preds_color:
            container_values[real_color]["coincidentes"] += 1

# Print the counters for each color
for color, data in container_values.items():
    matching = data["coincidentes"]
    total = data["total"]
    print(f"La clase {color_equivalence[str(color)]} tiene {matching} píxeles coincidentes de un total de {total} píxeles, es decir, un {round((matching / total)*100, 2)}%.")
    
    
iou, dice = calculate_metrics(colorPredsOriginal, colorMaskOriginal)
print("\n\n\nEl valor de IOU es de: " + str(iou))
print("\nEl valor de DICE es de: " + str(dice))